In [46]:
import numpy as np
import pandas as pd

from scipy import stats
import itertools
from sklearn import linear_model
from numpy import ones,vstack
from numpy.linalg import lstsq

import statsmodels.api as sm;

pd.options.display.max_columns = 150;

In [4]:
housing = pd.read_csv('../data/ames_housing_price_data_v2.csv', index_col = 0);

In [63]:
y = housing["SalePrice"];
ylog = housing["SalePrice_log"];

x = housing.drop(["SalePrice", "SalePrice_log"], axis = 1);
x = x.fillna(0);
# x = sm.add_constant(x);

# model = sm.OLS(y.astype(float), x.astype(float));
# results = model.fit();
# print(results.summary()); # How to print out the summary report

In [30]:
typedict = {#'PID' : 'nominal',
            'SalePrice' : 'Continuous',
            #Matt
            'LotFrontage' : 'Continuous', 
            'LotArea' : 'Continuous',
            'maybe_LotShape' : 'Nominal',
            'LandSlope' : 'Nominal', 
            'LandContour' : 'Nominal', 
            'maybe_MSZoning' : 'Nominal', 
            'Street_paved' : 'Nominal', 
            'Alley' : 'Nominal',
            'Neighborhood' : 'Nominal', 
            #'drop_LotConfig' : 'nominal', 
            #'drop_Condition1' : 'nominal', 
            #'drop_Condition2' : 'nominal',
            'Foundation' : 'Nominal',
            'Utilities' : 'Nominal',
            'Heating' : 'Nominal',
            'HeatingQC_nom' : 'Ordinal',
            'CentralAir' : 'Nominal',
            'Electrical' : 'Nominal',
            'HeatingQC_ord' : 'Ordinal',
            'LotShape_com' : 'Nominal',
            'MSZoning_com' : 'Nominal',
            #'LF_Normal' : 'nominal',
            'LF_Near_NS_RR' : 'Nominal',
            'LF_Near_Positive_Feature' : 'Nominal',
            'LF_Adjacent_Arterial_St' : 'Nominal',
            'LF_Near_EW_RR' : 'Nominal',
            'LF_Adjacent_Feeder_St' : 'Nominal',
            'LF_Near_Postive_Feature' : 'Nominal',
            'Heating_com' : 'Nominal',
            'Electrical_com' : 'Nominal',
            'LotConfig_com' : 'Nominal', 
            'LotFrontage_log' : 'Continuous',
            'LotArea_log' : 'Continuous',
            #Oren 
            'MiscFeature': 'Nominal',
            'Fireplaces': 'Discrete',
            'FireplaceQu': 'Ordinal',
            'PoolQC': 'Ordinal',
            'PoolArea': 'Continuous',
            'PavedDrive': 'Nominal',
            'ExterQual': 'Ordinal',
            'OverallQual': 'Ordinal',
            'maybe_OverallCond': 'Ordinal',
            'MiscVal': 'Continuous',
            'YearBuilt': 'Discrete',
            'YearRemodAdd': 'Discrete',
            'KitchenQual': 'Ordinal',
            'Fence': 'Ordinal',
            'RoofStyle': 'Nominal',
            'RoofMatl': 'Nominal',
            #'maybe_Exterior1st': 'Nominal',
            #'drop_Exterior2nd': 'Nominal',
            'maybe_ExterCond': 'Ordinal',
            'maybe_MasVnrType': 'Nominal',
            'MasVnrArea': 'Continuous',
            #Mo
            #Basement
            'BsmtQual_ord': 'Ordinal',
            'BsmtCond_ord': 'Ordinal',
            'BsmtExposure_ord': 'Ordinal',
            #'BsmtQual_ord_lin': 'Ordinal',
            #'BsmtCond_ord_lin': 'Ordinal',
            #'BsmtExposure_ord_lin': 'Ordinal',
            'TotalBsmtSF': 'Continuous',
            'BSMT_GLQ':'Continuous', 
            'BSMT_Rec':'Continuous',
            'maybe_BsmtUnfSF': 'Continuous',
            'maybe_BSMT_ALQ':'Continuous',
            'maybe_BSMT_BLQ':'Continuous', 
            'maybe_BSMT_LwQ':'Continuous', 
            #'drop_BsmtQual': 'Nominal',
            #'drop_BsmtCond': 'Nominal',
            #'drop_BsmtExposure': 'Nominal',
            #'drop_BsmtFinType1': 'Nominal',
            #'drop_BsmtFinSF1': 'Continuous',
            #'drop_BsmtFinType2': 'Nominal',
            #'drop_BsmtFinSF2': 'Continuous',
            #Deck
            'WoodDeckSF':'Continuous', 
            'OpenPorchSF':'Continuous', 
            'ScreenPorch':'Continuous',
            'maybe_EnclosedPorch':'Continuous',
            'maybe_3SsnPorch':'Continuous',
            #Garage
            'GarageFinish':'Nominal', 
            'GarageYrBlt':'Continuous',
            'GarageCars':'Ordinal',
            'GarageArea':'Continuous',
            'GarageType_con':'Nominal',
            'maybe_GarageQual':'Nominal', 
            'maybe_GarageCond':'Nominal',
            #'drop_GarageType':'Nominal',

            # Hao-Wei
            "SaleType": "Nominal",
            "BldgType": "Nominal",
            "Functional_ord": "Ordinal", # Changed from "Functional"
            "1stFlrSF": "Continuous",
            "2ndFlrSF": "Continuous",
            "maybe_LowQualFinSF": "Continuous", # Rejectable p-value
            "GrLivArea": "Continuous",
            "BsmtFullBath": "Discrete",
            "maybe_BsmtHalfBath": "Discrete", # Rejectable p-value
            "FullBath": "Discrete",
            "maybe_HalfBath": "Discrete",
            "BedroomAbvGr": "Discrete",
            "KitchenAbvGr": "Discrete",
            "TotRmsAbvGrd": "Discrete",
            "MoSold": "Discrete", # Rejectable p-value
            "YrSold": "Discrete", # Rejectable p-value
            ####### Below are columns created by myself #######
            #"Functional_dis": "Discrete", # Functional in a (Salvage) 0-7 (Full) scale.
            "1stFlrSF_log": "Continuous",
            "2ndFlrSF_log": "Continuous",
            "GrLivArea_log": "Continuous",
            "number_floors": "Discrete",
            "attic": "Ordinal",
            "PUD": "Nominal",
            #### Whose?
            "SaleCondition": "Nominal",
            "SalePrice_log": "Continuous",
            "MS_coded": "Nominal",
            "sold_datetime": "Discrete"
}

In [39]:
keys_list = list(typedict.keys());
house_list = housing.columns;

for k in keys_list:
    if typedict[k] not in ["Nominal", "Ordinal", "Discrete", "Continuous"]:
        print(k)

## Outlier issue

Just here to mark down Mo's outliers

In [42]:
df = housing;

In [43]:
df_outlier = df[(df.index==908154205) | (df.index==902207130)]

In [47]:
df_outlier

,GrLivArea,SalePrice,maybe_MSZoning,LotFrontage,LotArea,Street_paved,Alley,maybe_LotShape,LandContour,Utilities,LandSlope,Neighborhood,BldgType,OverallQual,maybe_OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,maybe_MasVnrType,MasVnrArea,ExterQual,maybe_ExterCond,Foundation,maybe_BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC_nom,CentralAir,Electrical,1stFlrSF,2ndFlrSF,maybe_LowQualFinSF,BsmtFullBath,maybe_BsmtHalfBath,FullBath,maybe_HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Fireplaces,FireplaceQu,GarageYrBlt,GarageFinish,GarageCars,GarageArea,maybe_GarageQual,maybe_GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,maybe_EnclosedPorch,maybe_3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice_log,HeatingQC_ord,LotShape_com,MSZoning_com,LF_Near_NS_RR,LF_Near_Positive_Feature,LF_Adjacent_Arterial_St,LF_Near_EW_RR,LF_Adjacent_Feeder_St,LF_Near_Postive_Feature,Heating_com,Electrical_com,LotConfig_com,LotFrontage_log,LotArea_log,BsmtCond_ord,BsmtQual_ord,BsmtExposure_ord,BSMT_GLQ,maybe_BSMT_ALQ,maybe_BSMT_BLQ,maybe_BSMT_LwQ,BSMT_Rec,GarageType_con,MS_coded,number_floors,attic,PUD,Functional_ord,1stFlrSF_log,2ndFlrSF_log,GrLivArea_log,sold_datetime
PID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
908154205,4676,184750,RL,130.0,40094,Pave,No alley access,IR1,Banked (rise from street level to building),EGWS,Gentle,Edwards,1Fam,10,5,2007,2008,Hip,Standard (Composite) Shingle,Stone,762.0,5,3,PConc,878.0,3138.0,GasA,Ex,Y,SBrkr,3138,1538,0,1.0,0.0,3,1,3,1,5,11,1,4,2007.0,Fin,3.0,884.0,TA,TA,Paved,208,406,0,0,0,0,NaN,No Fence,Nothing,0,10,2007,New,Partial,5.266584,5,Slightly irregular,"Residential, low-density",0,1,0,0,0,0,Gas-powered forced-air heating,"Standard circuit breakers, all Romex wiring",Inside lot (1 side frontage),2.113943,4.603079,3,5,4,2260.0,0.0,0.0,0.0,0.0,BuiltIn,2-Story,2,No attic,0,7,3.496653,3.187239,3.669875,2007-10-01
902207130,832,12789,RM,68.0,9656,Pave,No alley access,Reg,Level,EGWS,Gentle,OldTown,1Fam,2,2,1923,1970,Gable,Standard (Composite) Shingle,None,0.0,3,2,BrkTil,678.0,678.0,GasA,TA,N,SBrkr,832,0,0,0.0,0.0,1,0,2,1,3,5,1,4,1928.0,Unf,2.0,780.0,Fa,Fa,Dirt Gravel,0,0,0,0,0,0,NaN,No Fence,Nothing,0,6,2010,WD,Abnorml,4.106837,3,Regular,"Residential, medium-density",0,0,0,0,0,0,Gas-powered forced-air heating,"Standard circuit breakers, all Romex wiring",Inside lot (1 side frontage),1.832509,3.984797,2,2,1,0.0,0.0,0.0,0.0,0.0,Detchd,1-Story,1,No attic,0,7,2.920123,0.000000,2.920123,2010-06-01


## Imputing the data

In [66]:
x2 = pd.get_dummies(x, drop_first = True)

In [69]:
lm = LinearRegression();
lm.fit(x2, y);
lm.score(x2,y)


0.9280099223695875

In [87]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x2, y, test_size=0.3);

In [88]:
x_train.shape

(1806, 263)

In [118]:
from sklearn.pipeline import Pipeline;
from sklearn.preprocessing import StandardScaler;
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, SGDClassifier;
from sklearn.model_selection import KFold, GridSearchCV;


lasso = Lasso(max_iter=10000, tol = 2e-1);

pipe = Pipeline([ 
    ("standardize", StandardScaler()), ('linear_lasso', lasso)]);

params = {
    # "alpha": np.logspace(-6,3, num = 50)
    "alpha": [0.01, 0.1, 1, 10]
}

skfold = KFold(n_splits=5, shuffle=True, random_state=0);

gs = GridSearchCV(lasso, params, scoring="r2", cv=skfold);


gs.fit(x_train, y_train);

print("The best parameters are: ", gs.best_params_);


The best parameters are:  {'alpha': 10}


In [98]:
gs.score(x_test, y_test)

0.870474954204865

In [104]:
x_tm = np.matrix(x_test)

In [113]:
# (x_tm.T*x_tm).shape
x_test.cov()

,GrLivArea,LotFrontage,LotArea,OverallQual,maybe_OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,ExterQual,maybe_ExterCond,maybe_BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,maybe_LowQualFinSF,BsmtFullBath,maybe_BsmtHalfBath,FullBath,maybe_HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Fireplaces,FireplaceQu,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,maybe_EnclosedPorch,maybe_3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,HeatingQC_ord,LF_Near_NS_RR,LF_Near_Positive_Feature,LF_Adjacent_Arterial_St,LF_Near_EW_RR,LF_Adjacent_Feeder_St,LF_Near_Postive_Feature,LotFrontage_log,LotArea_log,BsmtCond_ord,BsmtQual_ord,BsmtExposure_ord,BSMT_GLQ,maybe_BSMT_ALQ,maybe_BSMT_BLQ,maybe_BSMT_LwQ,BSMT_Rec,number_floors,PUD,Functional_ord,1stFlrSF_log,2ndFlrSF_log,GrLivArea_log,maybe_MSZoning_C (all),maybe_MSZoning_FV,maybe_MSZoning_I (all),maybe_MSZoning_RH,maybe_MSZoning_RL,maybe_MSZoning_RM,Street_paved_Pave,Alley_No alley access,Alley_Paved,maybe_LotShape_IR2,maybe_LotShape_IR3,maybe_LotShape_Reg,LandContour_Depression (upward slope on both sides),LandContour_Hillside (downward slope on both sides),LandContour_Level,...,Heating_com_Gas-powered water/steam heating,Heating_com_Other,"Electrical_com_60 Amp fuse box, Romex or older wiring","Electrical_com_>60 Amp fuse box, all Romex wiring","Electrical_com_Standard circuit breakers, all Romex wiring",LotConfig_com_Corner lot,LotConfig_com_Cul-de-sac lot,LotConfig_com_Inside lot (1 side frontage),GarageType_con_Attchd,GarageType_con_BuiltIn,GarageType_con_Detchd,MS_coded_1.5-Story,MS_coded_2-Story,MS_coded_2.5-Story,MS_coded_2FamConv,MS_coded_Duplex,MS_coded_SplitMulti,MS_coded_SptFoyer,attic_No attic,attic_Unfinished,sold_datetime_2006-01-01,sold_datetime_2006-02-01,sold_datetime_2006-03-01,sold_datetime_2006-04-01,sold_datetime_2006-05-01,sold_datetime_2006-06-01,sold_datetime_2006-07-01,sold_datetime_2006-08-01,sold_datetime_2006-09-01,sold_datetime_2006-10-01,sold_datetime_2006-11-01,sold_datetime_2006-12-01,sold_datetime_2007-01-01,sold_datetime_2007-02-01,sold_datetime_2007-03-01,sold_datetime_2007-04-01,sold_datetime_2007-05-01,sold_datetime_2007-06-01,sold_datetime_2007-07-01,sold_datetime_2007-08-01,sold_datetime_2007-09-01,sold_datetime_2007-10-01,sold_datetime_2007-11-01,sold_datetime_2007-12-01,sold_datetime_2008-01-01,sold_datetime_2008-02-01,sold_datetime_2008-03-01,sold_datetime_2008-04-01,sold_datetime_2008-05-01,sold_datetime_2008-06-01,sold_datetime_2008-07-01,sold_datetime_2008-08-01,sold_datetime_2008-09-01,sold_datetime_2008-10-01,sold_datetime_2008-11-01,sold_datetime_2008-12-01,sold_datetime_2009-01-01,sold_datetime_2009-02-01,sold_datetime_2009-03-01,sold_datetime_2009-04-01,sold_datetime_2009-05-01,sold_datetime_2009-06-01,sold_datetime_2009-07-01,sold_datetime_2009-08-01,sold_datetime_2009-09-01,sold_datetime_2009-10-01,sold_datetime_2009-11-01,sold_datetime_2009-12-01,sold_datetime_2010-01-01,sold_datetime_2010-02-01,sold_datetime_2010-03-01,sold_datetime_2010-04-01,sold_datetime_2010-05-01,sold_datetime_2010-06-01,sold_datetime_2010-07-01
GrLivArea,2.587190e+05,3388.205030,1.417877e+06,418.992915,-81.486296,3740.104741,3341.201360,41740.097175,137.132660,-6.911279,63329.935392,1.083607e+05,1.129390e+05,141854.848025,3925.164806,22.778182,-4.026550,183.761109,107.960889,215.910692,8.235582,149.872725,639.539667,142.035049,424.530476,31099.377704,193.285072,54091.018277,19447.626689,11565.717170,-721.689220,-20.106727,3528.962484,1282.922531,-5993.387888,88.992434,-35.019194,158.871551,0.343016,8.609856,1.458783,-3.325085,-6.080538,2.144198,14.850172,45.007268,29.623996,176.533590,88.524234,72892.597040,-15345.819579,-7928.587307,3874.397741,-8461.821430,95.759740,-21.189160,-13.425374,38.108770,361.923041,70.251759,-4.251662,4.660760,-0.885842,-0.567601,27.194425,-26.150080,3.263937,3.139505,-1.596844,7.161843,4.603764,-51.126373,-0.606216,3.727835,-11.829327,...,2.590165,-2.215440,-6.276272,-18.207031,24.483303,6.975120,9.799467,-17.142182,33.

In [71]:
np.logspace(-6, 3, num=50)

array([1.00000000e-06, 1.52641797e-06, 2.32995181e-06, 3.55648031e-06,
       5.42867544e-06, 8.28642773e-06, 1.26485522e-05, 1.93069773e-05,
       2.94705170e-05, 4.49843267e-05, 6.86648845e-05, 1.04811313e-04,
       1.59985872e-04, 2.44205309e-04, 3.72759372e-04, 5.68986603e-04,
       8.68511374e-04, 1.32571137e-03, 2.02358965e-03, 3.08884360e-03,
       4.71486636e-03, 7.19685673e-03, 1.09854114e-02, 1.67683294e-02,
       2.55954792e-02, 3.90693994e-02, 5.96362332e-02, 9.10298178e-02,
       1.38949549e-01, 2.12095089e-01, 3.23745754e-01, 4.94171336e-01,
       7.54312006e-01, 1.15139540e+00, 1.75751062e+00, 2.68269580e+00,
       4.09491506e+00, 6.25055193e+00, 9.54095476e+00, 1.45634848e+01,
       2.22299648e+01, 3.39322177e+01, 5.17947468e+01, 7.90604321e+01,
       1.20679264e+02, 1.84206997e+02, 2.81176870e+02, 4.29193426e+02,
       6.55128557e+02, 1.00000000e+03])

In [65]:
from sklearn.linear_model import LinearRegression;

lm = LinearRegression();
lm.fit(x, y);
lm.score(x,y)
#plt.hist(residuals);

ValueError: could not convert string to float: 'RL'

## Preparing for statsmodel.api

In [5]:
import statsmodels.api as sm;
y = housing["SalePrice"];
ylog = housing["SalePrice_log"];
x = sm.add_constant(birthFeatures);
# Notice that the dependent variable (y) comes before the independent variables (x)
# Use df.columns.values to get all the column names
model = sm.OLS(np.log10(ncbirths["BirthWeightGm"]).astype(float), x.astype(float));
results = model.fit();
print(results.summary()); # How to print out the summary report

(2580, 97)